In [ ]:
from time import sleep

import gymnasium as gym
from ipycanvas import Canvas

env = gym.make('CartPole-v1',render_mode='rgb_array')
env.reset()

canvas = Canvas(width=400, height=600)
display(canvas)

for j in range(200):
    _, _, terminated, _, _ = env.step(j%2)
    canvas.put_image_data(env.render(), 0, 0)
    sleep(0.01)

In [ ]:
from math import pi

from ipywidgets import interact

from example_robot_data import load
from pinocchio.visualize import MeshcatVisualizer

In [ ]:
talos = load("talos")
viz = MeshcatVisualizer(talos.model, talos.collision_model, talos.visual_model)
viz.initViewer(loadModel=True)
viz.viewer.window.web_url = viz.viewer.window.web_url.replace("127.0.0.1", "hako.laas.fr")
viz.viewer.jupyter_cell()

In [ ]:
q0 = talos.q0
viz.display(q0)

In [ ]:
@interact(shoulder=(-0.5, 0.5, 0.01))
def say_hi(shoulder=0):
    q0[30] = pi - shoulder
    viz.display(q0)

## Possible alternative for gym display (through MP4)

#### Through MP4 embed
Second solution: the function below can render a video in the notebook.

*(taken from https://medium.com/@coldstart_coder/visually-rendering-python-gymnasium-in-jupyter-notebooks-4413e4087a0f)*

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

import io
import base64
from IPython import display
from IPython.display import HTML
# this function will take in the location of a video file and
# then use the virtual display to embed the video into the notebook
def embed_video(video_file):
    # open and read the raw data from the video
    video_data = io.open(video_file, 'r+b').read()
    # now we have to encode the data into base64 to work
    # with the virtual display
    encoded_data = base64.b64encode(video_data)
    # now we use the display.display function to take some html
    # and the encoded data and embed the html into the notebook!
    display.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded_data.decode('ascii'))))

Then run your environment in a RecordVideo wrapper.

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo

env = gym.make("CartPole-v1", render_mode="rgb_array")


In [ ]:
# By default, record only video of n**3 episods (when the episod is a natural cube). Change it to record the 20 first episods.
envr = RecordVideo(env, '/tmp/video', episode_trigger=lambda i: i<20)

In [ ]:
envr.reset()
for i in range(100):    # render the frame, this will save it to the video file
    envr.step(i % 2)
    envr.render()
envr.close() # Dont forget to close, then is encoded the video

In [ ]:
# Take care to read the video with the proper episod numero
embed_video("/tmp/video/rl-video-episode-2.mp4")